In [ ]:
from dhanhq import dhanhq
import yaml
import pandas  as pd
import numpy as np
from dhanhq import marketfeed



## Authentication

In [ ]:
CONFIG_FILE_PATH = r"..\\configs\\config.yaml" #This has the API key and client id
with open(CONFIG_FILE_PATH,'r') as config:
    api_config = yaml.safe_load(config)
APPLICAITON_ID = api_config.get('api_config')[0]
CLIENT_ID = api_config.get('api_config')[1]
ACCESS_TOKEN = api_config.get('api_config')[2]
EXP_HOURS = api_config.get('api_config')[3]
dhan = dhanhq(CLIENT_ID,ACCESS_TOKEN)

## Getting other variables from configuration

In [ ]:
OTHER_CONFIG_FILE_PATH = r"..\\configs\\other_config.yaml" # This we will configure all other variables
with open(OTHER_CONFIG_FILE_PATH,'r') as other_configs:
    gen_config = yaml.safe_load(other_configs)

In [ ]:
strike_size = gen_config.get('strike_size')
underlying = gen_config.get('index_file_metadata')
options = gen_config.get('options_file_metadata')

In [ ]:
underlying = pd.read_excel('underlying.xlsx')
options = pd.read_excel('Options.xlsx')


In [ ]:
UNDERLYING_INDEX = "NIFTY"

In [ ]:
def get_otms_itma(SPOT):
    INDEX_CE_ITM = {"ITM" :
    {
        x : int(SPOT - ((SPOT%strike_size[UNDERLYING_INDEX]) + (x*strike_size[UNDERLYING_INDEX]))) for x in range(10)
    }
    }
    INDEX_PE_ITM =  {"ITM" :
    {
        x : int(SPOT - ((SPOT%strike_size[UNDERLYING_INDEX]) - ((x+1)*strike_size[UNDERLYING_INDEX]))) for x in range(10)
    }
    }
    
    INDEX_ATM = SPOT - (SPOT%strike_size[UNDERLYING_INDEX])
    return({"CE" : INDEX_CE_ITM, "PE" : INDEX_PE_ITM, "ATM":INDEX_ATM})
    

In [41]:
def get_index_ltp(dhanO, INDEX):
    INDEX_ID = str(underlying.loc[(underlying.SEM_TRADING_SYMBOL == INDEX),['SEM_SMST_SECURITY_ID']].squeeze())
    INDEX_LTP = dhanO.ohlc_data(securities = {"IDX_I":[int(INDEX_ID)]})['data']['data']['IDX_I'][f'{INDEX_ID}']['last_price']
    STRIKE_PRICES = get_otms_itma(INDEX_LTP)
    return {"INDEX": INDEX, "SPOT": INDEX_LTP, "STRIKES":STRIKE_PRICES}

    

In [42]:
INDEX_LTP_STRIKE = get_index_ltp(dhan,UNDERLYING_INDEX)

In [ ]:
INDEX_ID = underlying.loc[(underlying['SEM_TRADING_SYMBOL']==INDEX_LTP_STRIKE['INDEX']),'SEM_SMST_SECURITY_ID'].squeeze()

In [47]:
INDEX_LTP_STRIKE['STRIKES']['CE']['ITM'][0]

23300

In [ ]:
dhan.ohlc_data()

In [51]:
options[(options['SEM_STRIKE_PRICE']==INDEX_LTP_STRIKE['STRIKES']['CE']['ITM'][0]) & (options['SEM_OPTION_TYPE']=='CE') & (options['Series']=='C') ]

,SEM_SMST_SECURITY_ID,SEM_LOT_UNITS,SEM_CUSTOM_SYMBOL,SEM_STRIKE_PRICE,SEM_OPTION_TYPE,SEM_UNDERLYING,Series
921,43285,25,NIFTY 21 NOV 23300 CALL,23300,CE,NIFTY,C
